In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
from scipy import stats

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Replace with your client ID, client secret, and redirect URI
client_id = '6f11478dbb564508aeaf87955de22cb6'
client_secret = 'df29bc83680d45889368d115410d1c40'
redirect_uri = 'http://localhost:8888/files/Jupyter%20Notebook%20Projects/my_spotify/callback.html'
# Replace with your actual redirect uri
# note that this adress is the only one I have on my app in spotifiy set as uri. 
#if you are trying to use it you shuold make your own app or try to mimic this location on your computer
#use the file "callback.html" I have created as your URI 
scope = 'user-read-private user-read-email playlist-read-private user-top-read'
sp_oauth = SpotifyOAuth(client_id=client_id,
                        client_secret=client_secret,
                        redirect_uri=redirect_uri,
                        scope=scope)

# Get authorization URL
auth_url = sp_oauth.get_authorize_url()
print("Please navigate here to authorize the application: ", auth_url)

# Manually get the authorization code after user authentication
# This should be obtained by redirecting the user to auth_url and handling the callback
authorization_code = input("Enter the authorization code: ")

# Exchange authorization code for access token
access_token = sp_oauth.get_access_token(code=authorization_code, as_dict=False)


Please navigate here to authorize the application:  https://accounts.spotify.com/authorize?client_id=6f11478dbb564508aeaf87955de22cb6&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Ffiles%2FJupyter%2520Notebook%2520Projects%2Fmy_spotify%2Fcallback.html&scope=user-read-private+user-read-email+playlist-read-private+user-top-read


Enter the authorization code:  AQBfWN34Elf3rczFlQX477h_lNcQduBO7esBlp3xc3Ve-ant6YMR2AUmiKpzd52gKb2ZAOwDT-5WFrz1R4CNBcoYyvY4WVo90zg_DeH0AYi2jFRkSTagc1pZ3NXJWPmFPKXHTulAAUnH1qYjccP8DGmFZLliddY6ckDykoue6xZnhQPa7-bAmhNCNaKE4SGuP36ymozNNCsqH5abcuZzOxdg3gKxkOfL-LcqWXtRCWLdMCVN6tFU6fhSSitQkX4JziAia551gQu2MhwF9mTERmXuED8cLX52-Law2IGjZD49V_Gw3yEuUClbK4S7uFmE3NoMT7qHLEq4C2UCO_tcDw


In [3]:
# Create a Spotify object with the access token
spotify = spotipy.Spotify(auth=access_token)
user_profile = spotify.current_user()

In [4]:
#getting my top 50 tracks into a list and adding the artists of thoes tracks into the artist set
my_top_tracks = spotify.current_user_top_tracks(limit=50,time_range="long_term")
my_top_tracks_list =[]
artists_set = set()
for song in my_top_tracks["items"]:
    my_top_tracks_list.append((song["name"],song["id"],song["popularity"],int(song["album"]["release_date"].split('-')[0])))
    artists_set.update((artist["name"], artist["id"],song["id"]) for artist in song["artists"])

In [5]:
#creating data frame from the set using pandas
my_top_tracks_df = pd.DataFrame(my_top_tracks_list,columns=["song_name","song_id","popularity","release_year"])

In [6]:
#getting my top 50 artists
my_top_artists = spotify.current_user_top_artists(limit=50, time_range="long_term")["items"]
my_top_artists_list = []
for artist in my_top_artists:
  my_top_artists_list.append((artist["name"], artist["id"]))

In [7]:
#creating a data frame from the set
my_top_artists_df = pd.DataFrame(my_top_artists_list,columns=["artist_name","artist_id"])

In [9]:
#getting 250 songs and their artists from each year between 1950-2024
song_name=[]
song_id=[]
popularity=[]
years = []
for year in range(1950,2025):
    for i in range(5):
        tracks = spotify.search(q=f"year:{year}",limit=50, type="track",offset=i*50)
        for song in tracks['tracks']['items']:
            song_name.append(song['name'])
            song_id.append(song['id'])
            popularity.append(song["popularity"])
            years.append(year)
            artists_set.update((artist["name"], artist["id"],song["id"]) for artist in song["artists"])

In [10]:
#creating a data frame with all of the artists so far and the connection to the songs
artists_songs_df = pd.DataFrame(artists_set,columns=["artist_name","artist_id","song_id"])

In [11]:
##creating a data frame with all of the artists so far
artists_df = pd.concat([artists_songs_df.drop(columns="song_id").drop_duplicates("artist_id"),my_top_artists_df],ignore_index=True)
artists_df.drop_duplicates("artist_id",inplace=True)
artists_songs_df.drop(columns="artist_name",inplace=True)

In [12]:
#reordernig the index
artists_df = artists_df.reset_index(drop=True)
artists_df.index = range(len(artists_df))

In [13]:
#create a data frame for the songs I got
tracks_df=pd.DataFrame({"song_name":song_name,"song_id":song_id,"popularity":popularity,"release_year":years})

In [14]:
#adding my top songs to the tracks data frame
tracks_df=pd.concat([tracks_df,my_top_tracks_df],ignore_index=True)

In [15]:
#droping duplicates that may occur because of my top tracks
tracks_df=tracks_df.drop_duplicates("song_id")
#reordernig the index
tracks_df = tracks_df.reset_index(drop=True)
tracks_df.index = range(len(tracks_df))

In [16]:
#getting audio features info for each song into a list
songs_features_list=[]
for i in range(0,len(tracks_df),100):
    end = min(i + 100, len(tracks_df))
    song_ids = tracks_df['song_id'].iloc[i:end].tolist()
    songs_features_list.extend(spotify.audio_features(song_ids))

In [17]:
#removing empty dict i got from the api
filtered_list = [item for item in songs_features_list if item is not None]

In [18]:
#getting explicit info for each song into a set
songs_info_set=set()
for i in range(0,len(tracks_df),50):
    end = min(i + 50, len(tracks_df))
    song_ids = tracks_df['song_id'].iloc[i:end].tolist()
    songs_info_set.update((track["explicit"],track["id"]) for track in spotify.tracks(song_ids)["tracks"])

In [19]:
len(songs_info_set)

18758

In [20]:
# Create a set of artist id to genres to approximate the songs genres
#also getting more info(popularity,followers) on the artists
artist_genres_set=set()
artist_pop_fol_set = set()
for i in range(0,len(artists_songs_df.drop_duplicates("artist_id")),50):
    end = min(i + 50, len(artists_songs_df.drop_duplicates("artist_id")))
    artists_ids = artists_songs_df.drop_duplicates("artist_id")["artist_id"].iloc[i:end].tolist()
    artists = spotify.artists(artists_ids)["artists"]
    artist_pop_fol_set.update((artist["id"],artist["popularity"],artist["followers"]["total"]) for artist in artists)
    artist_genres_set.update((artist["id"],tuple(artist["genres"])) for artist in artists)

In [21]:
my_artists = spotify.artists(my_top_artists_df["artist_id"].iloc[0:50].tolist())["artists"]

In [22]:
artist_pop_fol_set.update((artist["id"],artist["popularity"],artist["followers"]["total"]) for artist in my_artists)

In [23]:
artist_genres_set.update((artist["id"],tuple(artist["genres"])) for artist in my_artists)

In [24]:
artist_info_df = pd.DataFrame(artist_pop_fol_set,columns=["artist_id","artist_popularity","followers"])

In [25]:
#merging the new info
artists_df=artists_df.merge(artist_info_df,on="artist_id")

In [26]:
#creating an explicit data frame 
songs_explicit_df = pd.DataFrame(songs_info_set, columns=["explicit","song_id"])

In [27]:
#creating a audio features data frame 
audio_info_df = pd.DataFrame(filtered_list)

In [28]:
#merging into the tracks data frame with inner join
audio_info_df["song_id"]=audio_info_df["id"]
audio_info_df.drop(columns="id",inplace=True)
tracks_df=tracks_df.merge(audio_info_df,on="song_id")

In [29]:
tracks_df=tracks_df.merge(songs_explicit_df,on="song_id")

In [30]:
#removing unnecessary columns
tracks_df.drop(columns=["type","uri","track_href","analysis_url"],inplace=True)

In [31]:
#trying to simplfiy the genres of the songs to a main genre
def determine_genre(genres):
    keywords_map = {
        "pop": ["pop","chanson", "synthpop", "electropop", "k-pop", "teen pop", "pop rock", "pop punk", "australian pop"],
        "rock": ["rock","surf", "hard rock","progressive rock", "classic rock", "alternative rock", "indie rock", "punk rock", "garage rock","rock n roll","instrumental"],
        "hip_hop": ["hip hop","trip-hop", "rap", "trap", "boom bap", "gangsta rap","lo-fi", "lo-fi hip hop", "conscious hip hop", "pinoy trap"],
        "country": ["country","nashville", "bluegrass", "honky-tonk", "alt-country", "country rock","american" "americana","westren"],
        "jazz": ["jazz","lounge","hard bop", "bebop", "swing", "smooth jazz", "fusion", "free jazz", "big band"],
        "electronic": ["electronic", "EDM", "house", "techno", "trance", "dubstep", "drum and bass", "electro","slap house"],
        "classical": ["classical","orchestra","violin","choir", "baroque", "romantic", "symphony", "opera", "chamber music", "choral"],
        "reggae": ["reggae", "dub", "ska", "roots reggae", "dancehall", "rocksteady"],
        "blues": ["blues", "delta blues", "chicago blues", "electric blues", "country blues", "soul blues"],
        "metal": ["metal", "heavy metal", "thrash metal", "death metal", "black metal", "power metal", "doom metal","grindcore","death","doom"],
        "folk": ["folk", "contemporary folk","disco", "folk rock", "indie folk", "neofolk", "traditional folk", "folk-pop"],
        "punk": ["punk", "punk rock","trash", "hardcore punk", "post-punk", "pop punk", "garage punk"],
        "indie": ["indie", "indie pop", "indie rock", "indie folk", "alternative", "dream pop", "shoegaze", "australian indie"],
        "movie": ["broadway","ost","cartoon","soundtrack","cabaret","compositional ambient","video game","festigal", "tin pan alley","soundtracks","movie","hollywood"],
        "RnB": ["r&b","doo wop","funk","alternative r&b", "rhythm and blues", "neo soul", "contemporary r&b", "soul","doo wop","rhythm n blues","rnb","rhythm 'n' blues"],
        "mediterranean": ["mediterranean","muzika","italian","samba", "greek pop", "italian pop", "spanish pop", "turkish pop","israeli"],
        "adult_standards": ["adult standards", "easy listening", "vocal jazz", "swing revival", "crooners"]

    }
    genres = [genre.lower() for genre in genres]
    genre_count = defaultdict(int)
    
    for genre in genres:
        for main_genre, genre_keywords in keywords_map.items():
            if any(keyword in genre for keyword in genre_keywords):
                genre_count[main_genre] += 1
    if genre_count:
        return max(genre_count, key=genre_count.get)
    else:
        return "unknown"

In [32]:
artist_genres_set = {(artist,determine_genre(list(genres))) for artist,genres in dict(artist_genres_set).items()}

In [33]:
artist_genres_df = pd.DataFrame(artist_genres_set,columns=["artist_id","genre"])

In [34]:
tracks_df = tracks_df.merge(artist_genres_df.merge(artists_songs_df,on="artist_id").drop_duplicates("song_id").drop(columns=["artist_id"]),on="song_id")
#reordernig the index
tracks_df = tracks_df.reset_index(drop=True)
tracks_df.index = range(len(tracks_df))

In [35]:
#adding the new info to my top tracks data frame
my_top_tracks_df=my_top_tracks_df.drop(columns=["song_name","popularity","release_year"]).merge(tracks_df,how="left",on="song_id")

In [36]:
#adding the new info to my top artists as well
my_top_artists_df=my_top_artists_df.drop(columns=["artist_name"]).merge(artists_df,how="left",on="artist_id")

Now I have decided to add more songs and artists to my database from datasets I found online.
Note that this data is relevant to 2021, unlike the data I got from the API.
However, I will need to do some work to ensure the data matches my format. Let's begin by getting the data:

In [37]:
tracks2_df = pd.read_csv(r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\tracks.csv")
artists2_df = pd.read_csv(r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\artists.csv")

The 'tracks2' dataset has an inconsistent release year format. It also contains the artists
and their IDs as lists, which I don't need, but this information will help me approximate the genres of the songs.
Let's make sure we can work with lists and apply the 'eval' function.

In [38]:
tracks2_df["artists"]=tracks2_df["artists"].apply(eval)
tracks2_df["id_artists"]=tracks2_df["id_artists"].apply(eval)

In [39]:
#explode the lists
song_to_artist_df=tracks2_df.explode(['artists', 'id_artists'])

In [40]:
#prepering to merge
artists2_df["id_artists"] = artists2_df["id"]
artists2_df.drop(columns=["id"],inplace=True)

In [41]:
#merge the artist info on the songs_to_artists. inner join - I only want songs that the artists exist in the database so I can approximate the genres
song_to_artist_df=song_to_artist_df.merge(artists2_df[["id_artists","genres"]],on="id_artists")

In [42]:
# I will leave the song_to_artist dataframe to add to the previuos one and create a new one for the next step
song_to_genres_df = song_to_artist_df.drop_duplicates("id")[["id","genres"]]

In [43]:
song_to_genres_df["genres"] =song_to_genres_df["genres"].apply(eval)

In [44]:
song_to_genres_df.set_index("id",inplace=True)

In [45]:
#my function works faster on a dict. to use 'apply' on a col took too long
song_to_genres_dic = song_to_genres_df.to_dict()["genres"]

In [46]:
#approximate the genres with my function
song_to_genres_dic = {(song,determine_genre(genres)) for song,genres in song_to_genres_dic.items()}

In [47]:
song_to_genres_df = pd.DataFrame(song_to_genres_dic,columns=["song_id","genre"])

In [48]:
#since I dont want to add more songs with unknown genres i will drop thoes
song_to_genres_df=song_to_genres_df[song_to_genres_df["genre"]!="unknown"]

In [49]:
#preparing to concat
tracks2_df["song_name"] = tracks2_df["name"]
tracks2_df["song_id"] = tracks2_df["id"]
tracks2_df["release_year"] = tracks2_df["release_date"].apply(lambda x: int(x[:4]))
tracks2_df["explicit"] = tracks2_df["explicit"].apply(lambda x: x==1)

In [50]:
tracks2_df=tracks2_df.drop(columns=["name","id","release_date","artists","id_artists"]).drop_duplicates("song_id")

In [51]:
#add the genre info with inner join
tracks2_df = tracks2_df.merge(song_to_genres_df,on="song_id")

In [52]:
#preparing to concat the artists
artists2_df["artist_id"]= artists2_df["id_artists"]
artists2_df["artist_name"] = artists2_df["name"]
artists2_df["artist_popularity"] = artists2_df["popularity"]

In [53]:
artists2_df.drop(columns=["name","id_artists","genres","popularity"],inplace=True)

In [54]:
#adding to my data frame
artists_df=pd.concat([artists_df,artists2_df],ignore_index=True)
artists_df.drop_duplicates("artist_id",inplace=True)

In [55]:
artists_df = artists_df.reset_index(drop=True)
artists_df.index = range(len(artists_df))

In [56]:
#adding to my data frame
tracks_df=pd.concat([tracks_df,tracks2_df],ignore_index=True)
tracks_df.drop_duplicates("song_id",inplace=True)

In [57]:
tracks_df = tracks_df.reset_index(drop=True)
tracks_df.index = range(len(tracks_df))

In [ ]:
song_to_artist_df = song_to_artist_df[["id","id_artists"]]
song_to_artist_df["artist_id"] = song_to_artist_df["id_artists"]
song_to_artist_df["song_id"] = song_to_artist_df["id"]
song_to_artist_df.drop(columns=["id_artists","id"],inplace=True)

In [60]:
song_to_artist_df.drop(columns=["id_artists","id"],inplace=True)

In [61]:
#filtering connections betweens songs and artists that the songs are not in the data base
song_to_artist_df = song_to_artist_df[song_to_artist_df['song_id'].isin(tracks2_df['song_id'])]

In [62]:
#adding to my data frame
artists_songs_df=pd.concat([artists_songs_df,song_to_artist_df[["artist_id","song_id"]]],ignore_index=True)
artists_songs_df.drop_duplicates(["artist_id","song_id"],inplace=True)

In [63]:
artists_songs_df = artists_songs_df.reset_index(drop=True)
artists_songs_df.index = range(len(artists_songs_df))

In [ ]:
tracks_df["duration_sec"] = tracks_df["duration_ms"]/1000
tracks_df.drop(columns="duration_ms",inplace=True)

In [72]:
my_top_artists_df.to_csv(path_or_buf=r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\my_top_artists_df.csv")
my_top_tracks_df.to_csv(path_or_buf=r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\my_top_tracks_df.csv")
tracks_df.to_csv(path_or_buf=r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\tracks_df.csv")
artists_df.to_csv(path_or_buf=r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\artists_df.csv")
artists_songs_df.to_csv(path_or_buf=r"C:\Users\Mbtpem\Jupyter Notebook Projects\my_spotify\artists_songs_df.csv")